## CONTENT BASED RECOMENDERATION SYSTEM MODEL BUILDING USING COSINE SIMILLARITY
> AUTHOR: YASHSHREE BAVISKAR

    TEAM: 14

In [5]:
import numpy as np # for numrical caluation analysis
import pandas as pd # for data wrangling
import warnings
warnings.filterwarnings('ignore')

In [8]:
df= pd.read_csv("Movies_Clean_data.csv")

In [9]:
df.head(5)

,Unnamed: 0,id,title,tags
0,0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,1,8844,Jumanji,When siblings Judy and Peter discover an encha...
2,2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,3,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,4,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...


In [10]:
#Importing Required libraries
import nltk
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
#Defining the stop words
stop_words = stopwords.words('english')

#Defining the lemmatizer
lemmatizer = WordNetLemmatizer()

In [13]:
#Function Definition for using regex operations and 
## other text preprocessing for getting cleaned texts
def clean_comments(text):
    
    #convert to lower case
    lowered_text = text.lower()
    
    #Removing Punctuations
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\_',' ',text)
    
    #Removing all the non-ascii characters 
    clean_words = re.sub(r'[^\x00-\x7f]',r'', text)
    
    #Removing the unwanted white spaces
    text = " ".join(text.split()) 
    
    #Splitting data into words
    tokenized_text = word_tokenize(text)
    
    #Removing remaining tokens that are not alphabetic, 
    #Removing stop words and Lemmatizing the text
    removed_stop_text = [lemmatizer.lemmatize(word) for word 
                         in tokenized_text 
                         if word not in stop_words if word.isalpha()]
   
    return " ".join(removed_stop_text)

In [14]:
>>> import nltk
>>> nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
>>> import nltk
>>> nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [16]:
>>> import nltk
>>> nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [17]:
# Calling the above function for the column comment_text in training dataset to replace original with cleaned text
df['tags'] = df['tags'].apply(clean_comments)
df['tags'].head()

0    Led Woody Andy toy live happily room Andy birt...
1    When sibling Judy Peter discover enchanted boa...
2    A family wedding reignites ancient feud next d...
3    Cheated mistreated stepped woman holding breat...
4    Just George Banks recovered daughter wedding r...
Name: tags, dtype: object

In [18]:
df['tags'] = df['tags'].str.lower()

In [19]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [20]:
tf = TfidfVectorizer(max_features=3000,stop_words='english')
#features = tf.fit_transform(df['tags'])

In [21]:
Count_Vector = tf.fit_transform(df['tags']).toarray()

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
C_Vector = CountVectorizer(max_features=3000,stop_words='english')

In [23]:
Count_Vector = C_Vector.fit_transform(df['tags']).toarray()

In [24]:
Count_Vector.shape

(18365, 3000)

#### Using cosine Similarity Building the model for `Content Based Recommendation System`

In [25]:
from sklearn.metrics.pairwise import cosine_similarity

In [26]:
similarity = cosine_similarity(Count_Vector)

In [27]:
similarity

array([[1.        , 0.04445542, 0.04783649, ..., 0.0347524 , 0.01843024,
        0.        ],
       [0.04445542, 1.        , 0.0733674 , ..., 0.07106691, 0.        ,
        0.        ],
       [0.04783649, 0.0733674 , 1.        , ..., 0.        , 0.02027768,
        0.        ],
       ...,
       [0.0347524 , 0.07106691, 0.        , ..., 1.        , 0.11785113,
        0.        ],
       [0.01843024, 0.        , 0.02027768, ..., 0.11785113, 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

##  Showing Recomendations of Movies

In [28]:
def recommend(movie):
    index = df[df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(df.iloc[i[0]].title)

In [29]:
recommend('Toy Story')

Toy Story 3
Toy Story 2
The Champ
The 40 Year Old Virgin
The Toy


In [30]:
recommend('The Wolfman')

An American Werewolf in London
An American Werewolf in Paris
Blood and Chocolate
Dark Wolf
Werewolf of London


In [31]:
import pickle

In [34]:
pickle.dump(similarity,open("similarity_tf.pkl", 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

In [35]:
import os

print(os.path.getsize("similarity_tf.pkl"))

2698185963


In [36]:
import os

print(os.path.getsize("similarity_tf.pkl"))

2698185963


In [37]:
import gzip

In [38]:
pickle.dump(similarity, gzip.open("similarity_gzip.pkl", 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

In [39]:
print(os.path.getsize("similarity_gzip.pkl"))

527803340


In [40]:
!pip install bz2file

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for bz2file: filename=bz2file-0.98-py3-none-any.whl size=6882 sha256=9c27e1dc7e8d0ffaf527c0f8781cf0f932e917debbb743ceb013669e8929202b
  Stored in directory: /root/.cache/pip/wheels/85/ce/8d/b5f76b602b16a8a39f2ded74189cf5f09fc4a87bea16c54a8b
Successfully built bz2file


### compressing the file

In [41]:
import bz2file as bz2

In [42]:
ofile = bz2.BZ2File("BinaryData",'wb')
pickle.dump(similarity,ofile)
ofile.close()
 
print(os.path.getsize("BinaryData"))

426177591


In [44]:
ifile = bz2.BZ2File("BinaryData",'rb')
newdata = pickle.load(ifile)
ifile.close()

In [45]:
import joblib

In [46]:
joblib.dump(similarity, "similarity_jlib")

['similarity_jlib']

------------------------------------------------------------------------------------------------------------------------

## References:
    
1. [Stopwords](http://127.0.0.1:8888/?token=00cc485df8bac526527b36628642f159eed2a4b961b29752)
2. [Lemmatization](https://en.wikipedia.org/wiki/Lemmatisation)
3. [TFIDF Vectorization](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)
4. [Cosine Similarity](https://en.wikipedia.org/wiki/Cosine_similarity)